# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

import nltk
nltk.download(['punkt', 'wordnet'])

import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer, precision_score, accuracy_score, recall_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
import pickle


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('messages_df', engine)
df['message'] = df['message'].astype('str')
X = df['message'] 
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)
category_names = y.columns

In [3]:
y.mean().sort_values(ascending=False)

related                   0.766387
aid_related               0.414095
weather_related           0.278304
direct_report             0.193430
request                   0.170512
other_aid                 0.131436
food                      0.111421
earthquake                0.093659
storm                     0.093201
shelter                   0.088159
floods                    0.082086
medical_help              0.079488
infrastructure_related    0.065126
water                     0.063751
other_weather             0.052559
buildings                 0.050840
medical_products          0.050076
transport                 0.045798
death                     0.045531
other_infrastructure      0.043965
refugees                  0.033384
military                  0.032811
search_and_rescue         0.027655
money                     0.023033
electricity               0.020321
cold                      0.020168
security                  0.017991
clothing                  0.015432
aid_centers         

In [4]:
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    clean_text = re.sub(r"[^a-zA-Z0-9]", " ", text).lower()
    tok = word_tokenize(clean_text)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w, pos='v') for w in tok]
        
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [8]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred = pipeline.predict(X_test)
y_pred_df = pd.DataFrame(y_pred)
y_pred_df.columns = y.columns
y_pred_df


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
8,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
for i in y_test.keys():
    print(' --------------------- '+i+' --------------------- ')
    print(classification_report(y_test[i], y_pred_df[i]))

 --------------------- related --------------------- 
             precision    recall  f1-score   support

          0       0.64      0.38      0.48      1550
          1       0.83      0.94      0.88      4995

avg / total       0.79      0.80      0.78      6545

 --------------------- request --------------------- 
             precision    recall  f1-score   support

          0       0.88      0.99      0.93      5411
          1       0.86      0.38      0.53      1134

avg / total       0.88      0.88      0.86      6545

 --------------------- offer --------------------- 
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6512
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6545

 --------------------- aid_related --------------------- 
             precision    recall  f1-score   support

          0       0.73      0.87      0.80      3855
          1       0.74      0.

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


 --------------------- cold --------------------- 
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      6401
          1       0.74      0.10      0.17       144

avg / total       0.97      0.98      0.97      6545

 --------------------- other_weather --------------------- 
             precision    recall  f1-score   support

          0       0.95      1.00      0.97      6192
          1       0.75      0.03      0.05       353

avg / total       0.94      0.95      0.92      6545

 --------------------- direct_report --------------------- 
             precision    recall  f1-score   support

          0       0.86      0.98      0.91      5288
          1       0.78      0.32      0.46      1257

avg / total       0.84      0.85      0.83      6545



In [11]:
print('Precision:', precision_score(y_test.values, y_pred, average='micro'))
print('Recall:', recall_score(y_test.values, y_pred, average='micro'))
print('F1 Score:', f1_score(y_test.values, y_pred, average='micro'))

Precision: 0.810927638973
Recall: 0.454633578907
F1 Score: 0.582626789652


### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
pipeline = Pipeline([('features', FeatureUnion([('text_pipeline', 
                                                 Pipeline([
                                                     ('vect', CountVectorizer(tokenizer=tokenize)),
                                                     ('tfidf', TfidfTransformer()),
                                                     
                                                 ])),
                                                
                                               ])),
                                                 ('clf', MultiOutputClassifier(RandomForestClassifier()))
                                               ])


In [13]:
parameters = {'clf__estimator__n_estimators': [50, 100, 150]}

In [14]:
cv = GridSearchCV(pipeline, param_grid=parameters, scoring = make_scorer(f1_score, average='micro'))

In [15]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [50, 100, 150]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(f1_score, average=micro), verbose=0)

In [16]:
y_pred = cv.predict(X_test)
y_pred_df = pd.DataFrame(y_pred)
y_pred_df.columns = y.columns
y_pred_df


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
8,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
for i in y_test.keys():
    print(' --------------------- '+i+' --------------------- ')
    print(classification_report(y_test[i], y_pred_df[i]))

 --------------------- related --------------------- 
             precision    recall  f1-score   support

          0       0.76      0.29      0.42      1550
          1       0.81      0.97      0.89      4995

avg / total       0.80      0.81      0.78      6545

 --------------------- request --------------------- 
             precision    recall  f1-score   support

          0       0.89      0.99      0.94      5411
          1       0.90      0.43      0.58      1134

avg / total       0.89      0.89      0.88      6545

 --------------------- offer --------------------- 
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6512
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6545

 --------------------- aid_related --------------------- 
             precision    recall  f1-score   support

          0       0.77      0.89      0.83      3855
          1       0.79      0.

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [18]:
print('Precision:', precision_score(y_test.values, y_pred, average='micro'))
print('Recall:', recall_score(y_test.values, y_pred, average='micro'))
print('F1 Score:', f1_score(y_test.values, y_pred, average='micro'))

Precision: 0.833856809213
Recall: 0.501841799147
F1 Score: 0.626584768072


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [19]:
pipeline = Pipeline([('features', FeatureUnion([('text_pipeline', 
                                                 Pipeline([
                                                     ('vect', CountVectorizer(tokenizer=tokenize)),
                                                     ('tfidf', TfidfTransformer()),
                                                     
                                                 ])),
                                                
                                               ])),
                                                 ('clf', MultiOutputClassifier(AdaBoostClassifier()))
                                               ])


In [20]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))])

In [21]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [22]:
y_pred = pipeline.predict(X_test)
y_pred_df = pd.DataFrame(y_pred)
y_pred_df.columns = y.columns
y_pred_df


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
1,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
8,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
for i in y_test.keys():
    print(' --------------------- '+i+' --------------------- ')
    print(classification_report(y_test[i], y_pred_df[i]))

 --------------------- related --------------------- 
             precision    recall  f1-score   support

          0       0.66      0.38      0.48      1550
          1       0.83      0.94      0.88      4995

avg / total       0.79      0.81      0.79      6545

 --------------------- request --------------------- 
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      5411
          1       0.78      0.54      0.64      1134

avg / total       0.89      0.89      0.89      6545

 --------------------- offer --------------------- 
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6512
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6545

 --------------------- aid_related --------------------- 
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3855
          1       0.75      0.

In [24]:
print('Precision:', precision_score(y_test.values, y_pred, average='micro'))
print('Recall:', recall_score(y_test.values, y_pred, average='micro'))
print('F1 Score:', f1_score(y_test.values, y_pred, average='micro'))

Precision: 0.776961103161
Recall: 0.578955021326
F1 Score: 0.66350052769


### 9. Export your model as a pickle file

In [25]:
with open("classifier.pickle", "wb") as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.